In [1]:
import merge_utils as mu
import write_utils as wu
import parse_utils as pu
from operator import itemgetter

In [2]:
def recommendation_list(string):
    lista=[]
    split = string.split(',')
    for i in split:
        if i != '':
            lista.append((int(i)))
            
    return lista

def parse_submission(sc, path):
    rdd = sc.textFile(path)
    
    header = rdd.first()

    rdd = (rdd.filter(lambda x: x != header)
    .map(lambda x: x.split("\t"))
    .filter(lambda x: x[0] != '')
    .map(lambda x:(int(x[0]),recommendation_list(x[1])))
    )

    return rdd

def recommendation_list_with_value(string, value):
    updating_value = value
    lista=[]
    split = string.split(',')
    for i in split:
        if i != '':
            updating_value -= 0.0001
            lista.append((int(i),updating_value))
            
    return lista

def psv(sc, path, value):
    rdd = sc.textFile(path)
    
    header = rdd.first()

    rdd = (rdd.filter(lambda x: x != header)
    .map(lambda x: x.split("\t"))
    .filter(lambda x: x[0] != '')
    .map(lambda x:(int(x[0]),recommendation_list_with_value(x[1], value)))
    )

    return rdd

def rdd_no_header(rdd):
    header = rdd.first()
    return rdd.filter(lambda x: x!=header)

def get_sim_dict(sim_rdd):
    return (rdd_no_header(sim_rdd)
                .map(lambda x:x.split("\t"))
                .filter(lambda x:x[0]!='' or x[1]!='')
                .map(lambda x:(int(x[0]),from_string_to_list(x[1], 30)))
               )

def get_weight_value(jobs, value):
    if len(jobs) > 0:
        return [(i[0],i[1]) for i in jobs[:50]]
    else:
        return []
    
def psvs(sc, path, value):
    rdd = sc.textFile(path)
    rec_rdd = (get_sim_dict(rdd)
               .map(lambda x: (x[0], get_weight_value(x[1], value)))
               .filter(lambda x: len(x[1]) > 0))
    
    return rec_rdd

def from_string_to_list(stringa,kNN=30):
    lista_stringa = stringa.replace("[","").replace("]","").replace("(","").split("),")
    lista_stringa = [i.replace(")","").split(", ") for i in lista_stringa]
    final=[]
    for tup in lista_stringa:
        if tup[0]!='':# or tup[1]!='':
            final.append((int(tup[0]),float(tup[1])))
    
    return sorted(final,key=itemgetter(1),reverse=True)[:kNN]

def filter_data(x):
    user = x[0]
    jobs = x[1]
    
    to_be_filter = union_dict.get(user)
    if to_be_filter != None:
        return [j for j in jobs if int(j[0]) not in to_be_filter]
    else:
        return jobs

In [4]:
interactions_rdd = (pu.parseInteractions(sc, "../../Dataset/interactions.csv")
                  .map(lambda x: (x.userId, [x.itemId])).reduceByKey(lambda x,y: x+y)
                  .map(lambda x: (x[0], list(set(x[1])))))
print interactions_rdd.take(1)

impressions_rdd = (pu.parseImpressions(sc, "/Users/tommy/Downloads/impressions.csv")
                  .map(lambda x: (x.userId, x.itemIds))
                  .map(lambda x: (x[0], list(set(x[1])))))
print impressions_rdd.take(1)

union_rdd = impressions_rdd.union(interactions_rdd).reduceByKey(lambda x,y: x+y)
print union_rdd.take(1)

union_dict = dict(union_rdd.collect())

[(2818048, [161771])]
[(621311, [1037920, 2382594, 1515140, 2501, 2606598, 2163175, 143112, 653321, 1578157, 1710350, 1587453, 968753, 2190578, 2270308, 239162, 1661883, 2844858, 306686, 1222303])]
[(2506752, [90120, 1053452, 950420, 1423129, 1754395, 996894, 820002, 693415, 2486575, 1171976, 938290, 164276, 348599, 1755963, 854206, 2773055, 2715712, 1827778, 811077, 581327, 2834514, 926933, 934872, 521945, 1356639, 784737, 2117988, 1108837, 1982203, 166784, 1754884, 1171976, 1842710, 1423129, 1078303, 693415, 938290, 2138292, 2547004, 1232957, 1487551, 1827778, 78669, 581327, 2834514, 2080350, 1356639, 2117988, 1108837, 2773055, 2287467, 838894, 1054322, 2383094, 1004665, 1982203, 1366336, 213441, 1244196, 1190437, 2235910, 2071368, 277062, 715276, 572690, 270559, 2486575, 2834514, 959545, 1092821, 1058425, 1194436, 1920047, 1206685, 1666847, 2146624, 648769, 79531, 619366, 277062, 1954727, 2071368, 2235910, 2799947, 78669, 1636910, 2486575, 953872, 2350577, 2834514, 926933, 348599, 4

In [8]:
#submission_name = "../../Submissions/ens_subs/tf_ervin_95k.csv"
submission_name = "/Users/tommy/Downloads/TFIDFMarco_noint.csv"
submission_rdd = psv(sc, submission_name, 1).map(lambda x: (x[0], x[1][:50]))

In [9]:
recommendation_rdd = submission_rdd.map(lambda x: (x[0], filter_data(x)))

In [10]:
wu.prepareSubmission(recommendation_rdd, "../../Submissions/ens_subs/new_idf")

Submission file ../../Submissions/ens_subs/new_idf created successfully
